In [ ]:
import kagglehub
import os
import json
import numpy as np
import cv2
from tqdm import tqdm
import ultralytics
from ultralytics import YOLO


In [2]:
kaggle_download_path = kagglehub.dataset_download("google/tinyquickdraw")
print("Downloaded path:", kaggle_download_path)

Downloaded path: C:\Users\gabri\.cache\kagglehub\datasets\google\tinyquickdraw\versions\3


In [3]:
downloaded_files = os.listdir(kaggle_download_path)
print("Downloaded files:", downloaded_files)

Downloaded files: ['baseball bat.ndjson', 'finger.ndjson', 'helmet.ndjson', 'hexagon.ndjson', 'hockey stick.ndjson', 'quickdraw_simplified', 'sketches']


In [ ]:
# Defining dataset paths
ndjson_path = kaggle_download_path  # The directory containing .ndjson files
output_path = "C:/MSAAI/AAI-590/Data/tinyquickdraw_dataset"  

In [ ]:
# Creating output directories
os.makedirs(output_path, exist_ok=True)
train_path = os.path.join(output_path, "train")
valid_path = os.path.join(output_path, "valid")

os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)

In [ ]:
# Function to draw strokes and save as PNG
def draw_doodle(strokes, filename, save_path, img_size=256):
    img = np.ones((img_size, img_size), dtype=np.uint8) * 255  # White background
    for stroke in strokes:
        for i in range(len(stroke[0]) - 1):
            cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i+1], stroke[1][i+1]), 0, 2)
    
    # Resizing for YOLO
    img = cv2.resize(img, (64, 64)) 
    cv2.imwrite(os.path.join(save_path, filename), img)

In [ ]:
# Processing each .ndjson file
for file in tqdm(os.listdir(ndjson_path)):
    if file.endswith(".ndjson"):
        label = file.replace(".ndjson", "")  # Using filename as class label
        label_path = os.path.join(train_path, label)
        os.makedirs(label_path, exist_ok=True)

        with open(os.path.join(ndjson_path, file), "r") as f:
            drawings = [json.loads(line) for line in f]

        # Saving first 80% to train, last 20% to validation
        num_train = int(len(drawings) * 0.8)
        
        for i, drawing in enumerate(drawings):
            filename = f"{label}_{i}.png"
            save_dir = train_path if i < num_train else valid_path
            save_label_path = os.path.join(save_dir, label)
            os.makedirs(save_label_path, exist_ok=True)
            draw_doodle(drawing["drawing"], filename, save_label_path)

print("NDJSON to PNG conversion completed!")

100%|██████████| 7/7 [05:29<00:00, 47.00s/it]

NDJSON to PNG conversion completed!


Defining YOLO Dataset config:

In [ ]:
import glob

# YOLO annotation format: <class_id> <x_center> <y_center> <width> <height>
def create_yolo_labels(image_folder, label_folder, class_mapping):
    os.makedirs(label_folder, exist_ok=True)
    images = glob.glob(os.path.join(image_folder, "*/*.png"))

    for img_path in tqdm(images):
        img_name = os.path.basename(img_path)
        class_name = os.path.basename(os.path.dirname(img_path))
        class_id = class_mapping[class_name]

        label_path = os.path.join(label_folder, img_name.replace(".png", ".txt"))
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1 1\n")  # Bounding box: whole image

# Creating class mapping
class_names = sorted([f.replace(".ndjson", "") for f in os.listdir(ndjson_path) if f.endswith(".ndjson")])
class_mapping = {name: i for i, name in enumerate(class_names)}

# Generating labels
create_yolo_labels(train_path, os.path.join(output_path, "train_labels"), class_mapping)
create_yolo_labels(valid_path, os.path.join(output_path, "valid_labels"), class_mapping)

print("YOLO annotations created successfully!")

100%|██████████| 137243/137243 [00:28<00:00, 4735.53it/s]

YOLO annotations created successfully!


In [ ]:
data_yaml = f"""train: {output_path}/train
val: {output_path}/valid

nc: {len(class_names)}  # Number of classes
names: {class_names}
"""

# Saving data.yaml
with open("data.yaml", "w") as f:
    f.write(data_yaml)

print("Dataset configuration saved as data.yaml")



Dataset configuration saved as data.yaml


In [ ]:
print("CUDA Available:", torch.cuda.is_available())
print("Ultralytics Version:", ultralytics.__version__)

CUDA Available: True
Ultralytics Version: 8.3.64


In [ ]:
#lowering ultralytics version for compatability 
pip install ultralytics==8.3.60

   ---------------------------------------- 0.0/906.9 kB ? eta -:--:--
   --------------------------------------- 906.9/906.9 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.91
    Uninstalling ultralytics-8.3.91:
      Successfully uninstalled ultralytics-8.3.91
Note: you may need to restart the kernel to use updated packages.


In [20]:
train_path = "C:/MSAAI/AAI-590/Data/tinyquickdraw_dataset/train"
print("Train folder contents:", os.listdir(train_path))


Train folder contents: ['baseball bat', 'baseball bat.cache', 'finger', 'helmet', 'hexagon', 'hockey stick']


In [ ]:
# Loading the YOLOv8 nano model
model = YOLO("yolov8n.pt")

# Training the model 
model.train(
    data="data.yaml",    # dataset configuration file
    epochs=10,           
    imgsz=128,           
    batch=8,             
    device="cuda",       
    workers=0           # Using 0 workers to avoid multiprocessing issues on Windows
)


New https://pypi.org/project/ultralytics/8.3.91 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.64  Python-3.10.14 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=128, save=True, save_period=-1, cache=False, device=cuda, workers=0, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False,

train: Scanning C:\MSAAI\AAI-590\Data\tinyquickdraw_dataset\train\baseball bat.cache... 0 images, 548967 backgrounds, 0 corrupt: 100%|██████████| 548967/548967 [00:00<?, ?it/s]


WARNING  No labels found in C:\MSAAI\AAI-590\Data\tinyquickdraw_dataset\train\baseball bat.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: Scanning C:\MSAAI\AAI-590\Data\tinyquickdraw_dataset\valid\baseball bat.cache... 0 images, 137243 backgrounds, 0 corrupt: 100%|██████████| 137243/137243 [00:00<?, ?it/s]

WARNING  No labels found in C:\MSAAI\AAI-590\Data\tinyquickdraw_dataset\valid\baseball bat.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to c:\Users\gabri\runs\detect\train13\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 128 train, 128 val
Using 0 dataloader workers
Logging results to c:\Users\gabri\runs\detect\train13
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10     0.197G          0     0.0684          0          0        128: 100%|██████████| 68621/68621 [1:24:50<00:00, 13.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8578/8578 [09:49<00:00, 14.55it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors